In [1]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import time
import pickle

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import lightgbm as lgb

from datetime import datetime, timedelta


from baseFunctions import *
from data_helpers import processData6, featureEngineering, getSequencesFast, removeOutliers


In [2]:
data, propDicts, flippedPropDicts = processData6()
data, timeFeatures = featureEngineering(data)

(3029400, 7) (3029400, 11)
dcoilwtico    529
dtype: int64
date          0
dcoilwtico    0
dtype: int64
(3029400, 12) (3029400, 11)


/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

(3029400, 12) (3029400, 15) data2


/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

(3029400, 18) (3029400, 15) data3


/Users/matthiaskargl/Codes/kaggle/store-sales-time-series-forecasting/analysis/data_helpers.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(3029400, 18) (3029400, 21)
finished data4
(3029400, 21) (3029400, 15)
finished data5


In [3]:
flippedPropDicts['family']

{0: 'AUTOMOTIVE',
 1: 'BABY CARE',
 2: 'BEAUTY',
 3: 'BEVERAGES',
 4: 'BOOKS',
 5: 'BREAD/BAKERY',
 6: 'CELEBRATION',
 7: 'CLEANING',
 8: 'DAIRY',
 9: 'DELI',
 10: 'EGGS',
 11: 'FROZEN FOODS',
 12: 'GROCERY I',
 13: 'GROCERY II',
 14: 'HARDWARE',
 15: 'HOME AND KITCHEN I',
 16: 'HOME AND KITCHEN II',
 17: 'HOME APPLIANCES',
 18: 'HOME CARE',
 19: 'LADIESWEAR',
 20: 'LAWN AND GARDEN',
 21: 'LINGERIE',
 22: 'LIQUOR,WINE,BEER',
 23: 'MAGAZINES',
 24: 'MEATS',
 25: 'PERSONAL CARE',
 26: 'PET SUPPLIES',
 27: 'PLAYERS AND ELECTRONICS',
 28: 'POULTRY',
 29: 'PREPARED FOODS',
 30: 'PRODUCE',
 31: 'SCHOOL AND OFFICE SUPPLIES',
 32: 'SEAFOOD'}

In [4]:
trainF = [
       'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       'city', 'state', 'type', 'cluster', 'dcoilwtico', 'holidayType',
       'description', 'transferred', 
       #'transactions', 
       'store_closed',
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0',
       'day_of_year_f24_0', 'day_of_year_f52_0', 'weekday', 'month'
       ]

trainF2 = trainF + ['sales']



n_predictedValues = 16
look_back = 100
zScoreNorm = True

# Date string
date_string = "2017-05-01"
date_object = datetime.strptime(date_string, '%Y-%m-%d')
days_ago = date_object - timedelta(days=(look_back + n_predictedValues -1 +2))
days_ago_string = days_ago.strftime('%Y-%m-%d')
days_ago_string

maskTrain = data.date < date_string
maskTest = data.date  > days_ago_string #"2017-03-03" #42days + 15 day between (15 because we only want to iterate one value more from the test set)

log = {}

train = data.loc[data.dataT == 'train']

    

    
stdDict = {}
meanDict= {}
X_train = []
X_test  = []
y_train = []
y_test  = []
std_train, std_test = [], []
mean_train, mean_test = [], []
init= False
for familyId in data.family.unique():
       # start with only some families
       if familyId > 8:
          continue

       print(familyId)
       familyDf = train.loc[train.family==familyId]  

       for storeId in data.store_nbr.unique():
              storeDf = familyDf.loc[(familyDf.store_nbr == storeId)] 
              X_train0,y_train0,mean,std = getSequencesFast(storeDf.loc[maskTrain], trainF, look_back, n_predictedValues, zScoreNorm=zScoreNorm)
              X_test0, y_test0           = getSequencesFast(storeDf.loc[maskTest], trainF, look_back, n_predictedValues, zScoreNorm=False, applyZScoreNorm=True, meanZ=mean, stdZ=std)       
              stdDict[storeId] = std
              meanDict[storeId] = mean
              if init:
                     X_train = np.concatenate((X_train, X_train0), axis=0)
                     X_test  = np.concatenate((X_test,  X_test0), axis=0)
                     y_train = np.concatenate((y_train, y_train0), axis=0)
                     y_test  = np.concatenate((y_test,  y_test0), axis=0)
                     std_train = np.concatenate((std_train, np.ones(y_train0.shape)*std), axis=0)
                     std_test  = np.concatenate((std_test,  np.ones(y_test0.shape)*std), axis=0)
                     mean_train = np.concatenate((mean_train, np.ones(y_train0.shape)*mean), axis=0)
                     mean_test = np.concatenate((mean_test, np.ones(y_test0.shape)*mean), axis=0)
              else:
                     X_train, X_test, y_train,y_test = X_train0, X_test0, y_train0, y_test0
                     std_train  =np.ones(y_train0.shape)*std
                     std_test   =np.ones(y_test0.shape)*std
                     mean_train =np.ones(y_train0.shape)*mean
                     mean_test  =np.ones(y_test0.shape)*mean
                     init=True 


 

0
1
2
3
4
5
6
7
8


In [5]:
np.savez('train_test_Sequences_100_16_StdNormalized_fam0-7.npz', arr1=X_train, arr2=y_train, arr3=std_train, arr4=X_test, arr5=y_test, arr6=std_test)

# LGBM

In [ ]:
# Set parameters for LGBM model
params = {
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'rmsle',  # Mean squared error
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}   

# Train the model
num_round = 10  
gbms = [lgb.train(params, lgb.Dataset(X_train, label=y_train[:, i]),num_round, valid_sets=[lgb.Dataset(X_test, label=y_test[:,i])]) for i in range(y_train.shape[1])]   


forecast = np.column_stack([gbm.predict(X_train, num_iteration=gbm.best_iteration) for gbm in gbms])
if (forecast<0).any():
    print('negative values!!!')
    forecast = np.clip(forecast, 0, 1e29)
if zScoreNorm:
    forecast = forecast *std_train  + mean_train
    y_train = y_train *std_train + mean_train
rmsleTrain = np.sqrt(mean_squared_log_error(forecast, y_train))
forecast = np.column_stack([gbm.predict(X_test, num_iteration=gbm.best_iteration) for gbm in gbms])
if zScoreNorm:
    forecast = forecast *std_test  + mean_test
    y_test = y_test*std_test + mean_test
rmsleTest = np.sqrt(mean_squared_log_error(forecast, y_test))
print('familyId:', familyId, 'errors:  ', round(rmsleTrain,3), round(rmsleTest,3), y_train.shape[0], y_test.shape[0])


# facebook prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
# Assuming df is a pandas dataframe with 'ds' and 'y' columns
m = Prophet()
m.fit(df)

# Make future dataframe for 16 periods
future = m.make_future_dataframe(periods=16)
forecast = m.predict(future)


# Transformer

idea: we could use positional embeddings to define the relation between store and product

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Transformer

In [ ]:
tf.keras.utils.set_random_seed(42)

# Define the input
inputs = Input(shape=(1,), dtype=tf.string)


# Create a Transformer layer
transformer_layer = tfa.layers.transformer.Transformer(num_layers=2, num_heads=2, 
                                                       dim_model=32, dim_feedforward=128)
transformer_outputs = transformer_layer(tokenized_inputs)

# Create a dense layer for the output
outputs = Dense(num_classes, activation='softmax')(transformer_outputs)

# Define the model
model = Model(inputs, outputs)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape_oldS,
    input_shape_newS,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape_oldS)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

## transformer from scratch

In [6]:
# 2d positional encoding (feature & position)
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [7]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, sequence_length, d_model):
    super().__init__()
    self.d_model = d_model #d_model or model depth = dimensionality of encoding vector
    # no need for an embedding layer since my input is a time series and not a word token
    #self.embedding = tf.keras.layers.Embedding(sequence_length, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=sequence_length, depth=d_model)

    #TODO implement an encoding for the store and family
    #self.store_fam_encoding

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]

    #x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    #x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [8]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

# only connect sequence to past & not to future
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [9]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

In [10]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [11]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, sequence_length, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        sequence_length=sequence_length, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [12]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [13]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, sequence_length,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(sequence_length=sequence_length,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [14]:
class Transformer(tf.keras.Model):
  def __init__(self, *, enc_dec_num_layers, input_seq_dim_enc, input_seq_dim_dec, num_heads, fully_connected_size,
               input_sequence_len, output_sequence_len, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=enc_dec_num_layers, d_model=input_seq_dim_enc,
                           num_heads=num_heads, dff=dff,
                           sequence_length=input_sequence_len,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=enc_dec_num_layers, d_model=input_seq_dim_dec,
                           num_heads=num_heads, dff=dff,
                           sequence_length=output_sequence_len,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(output_sequence_len)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.pt.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    dropout_rate=dropout_rate)

# GRU

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
tf.keras.utils.set_random_seed(42)

model = Sequential()
model.add(GRU(32, return_sequences=True, input_shape=(100, 20)))
model.add(GRU(32))
model.add(Dense(16))

model.compile(loss='rmse', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_test, y_test))


In [ ]:
tf.keras.utils.set_random_seed(42)

# Define an input sequence.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Create a GRU layer
encoder = GRU(latent_dim, return_state=True)

# Get the output and state
encoder_outputs, state_h = encoder(encoder_inputs)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Create another GRU layer
decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)

# Get the output and state
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=state_h)

# Create a dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# Get the final output
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# LSTM

In [ ]:
tf.keras.utils.set_random_seed(42)

# Define an input sequence.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# Create a LSTM layer
encoder = LSTM(latent_dim, return_state=True)

# Get the output and states
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# Create another LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

# Get the output and states
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Create a dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# Get the final output
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# XGBoost